In [2]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

# Scraping missing data
In this notebook, we are scrappping missing data for our data story: analysing the ideal actor profile for each genre over time.

The missing data is:
1. height of every actor
2. movie success metric: imdb review score
3. ...???

## Scraping the height of every actor

In [3]:
# import data sets (column names are based on readme file of data set publisher)
character_metadata = pd.read_csv('./data/character.metadata.tsv', sep='\t', names=[
    'Wikipedia movie ID',
    'Freebase movie ID',
    'Movie release date',
    'Character name',
    'Actor date of birth',
    'Actor gender',
    'Actor height (in meters)',
    'Actor ethnicity (Freebase ID)',
    'Actor name',
    'Actor age at movie release',
    'Freebase character/actor map ID',
    'Freebase character ID',
    'Freebase actor ID',
])

In [4]:
# determmine amount of actor whoe height is unknown
actors_unknown_height = character_metadata[np.isnan(character_metadata["Actor height (in meters)"])][["Actor name"]].drop_duplicates()
actors_unknown_height.head()

,Actor name
7,Richard Cetrone
9,Duane Davis
10,Lobo Sebastian
11,Rodney A. Grant
13,Rick Edelstein


In [5]:
actors_unknown_height.shape

(121417, 1)

In [6]:
# get imdb actor list (source: https://datasets.imdbws.com/)
actors_imdb = pd.read_csv('./data/name.basics.tsv/data.tsv', sep='\t')
actors_imdb.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0056404,tt0057345,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0080455,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"


In [7]:
actors_imdb.shape

(12081796, 6)

In [8]:
#filter for actors whose height is unknown
actors_unknown_height = actors_unknown_height.merge(actors_imdb, left_on="Actor name", right_on='primaryName', how="left")
actors_unknown_height.head()

# note: one actor has several imdb ids from time to time

,Actor name,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,Richard Cetrone,nm0149150,Richard Cetrone,1961,\N,"stunts,actor","tt3498820,tt3778644,tt0451279,tt2975590"
1,Duane Davis,nm0204523,Duane Davis,\N,\N,"actor,producer","tt0228333,tt0107889,tt0094721,tt0095742"
2,Duane Davis,nm12443572,Duane Davis,\N,\N,actor,tt13620374
3,Duane Davis,nm2483556,Duane Davis,\N,\N,actor,tt0816237
4,Duane Davis,nm2836184,Duane Davis,\N,\N,actor,tt1141256


In [9]:
actors_unknown_height.shape

(393614, 7)

In [10]:
# perform webscrape for height
base_url = "https://www.imdb.com/name/" #+ imdb_id + "/bio/" e. g.: "nm0000226/""







In [20]:
page = requests.get('https://www.imdb.com/name/nm0000226/bio/')
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find('table', attrs={'id':'overviewTable'})
print(table)

data = []
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
data

<table class="dataTable labelValueTable" id="overviewTable">
<tr class="even"><td class="label">Born</td>
<td>
<time datetime="1968-9-25">
<a href="/search/name?birth_monthday=9-25&amp;refine=birth_monthday">September 25</a>, 
     
<a href="/search/name?birth_year=1968">1968</a>
</time>
in <a href="/search/name?birth_place=Philadelphia,%20Pennsylvania,%20USA">Philadelphia, Pennsylvania, USA</a> </td>
</tr>
<tr class="odd">
<td class="label">Birth Name</td><td>Willard Carroll Smith</td>
</tr>
<tr class="even"><td class="label">Nicknames</td><td>
Fresh Prince<br/>Mr. July<br/>Will    </td></tr>
<tr class="odd"><td class="label">Height</td>
<td>
6' 1½" (1.87 m)    </td></tr>
</table>


[['Born',
  'September 25, \n     \n1968\n\nin\xa0Philadelphia, Pennsylvania, USA'],
 ['Birth Name', 'Willard Carroll Smith'],
 ['Nicknames', 'Fresh PrinceMr. JulyWill'],
 ['Height', '6\'\xa01½"\xa0(1.87\xa0m)']]

In [12]:
for imdb_actor_id in actors_unknown_height["nconst"] : 
    base_url = "https://www.imdb.com/name/"+imdb_actor_id+"/bio/"

TypeError: can only concatenate str (not "float") to str

## Scraping imdb score of every given movie